In [1]:
from pyspark import SparkContext
from scipy import sparse as sm
from sklearn.preprocessing import normalize
import numpy as np
import csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from scipy.stats import spearmanr
from scipy.stats import pearsonr as pears
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
import time
from sparsesvd import sparsesvd
import math as mt
sc = SparkContext.getOrCreate()

In [2]:
train_rdd = sc.textFile("data/train.csv")
icm_rdd = sc.textFile("data/icm_fede.csv")
test_rdd= sc.textFile("data/target_users.csv")

train_header = train_rdd.first()
icm_header = icm_rdd.first()
test_header= test_rdd.first()

train_clean_data = train_rdd.filter(lambda x: x != train_header).map(lambda line: line.split(',')).map(lambda x: (int(x[0]), int(x[1]), float(x[2])))
icm_clean_data = icm_rdd.filter(lambda x: x != icm_header).map(lambda line: line.split(',')).map(lambda x: (int(x[0]), int(x[1])))
test_clean_data= test_rdd.filter(lambda x: x != test_header).map(lambda line: line.split(','))

test_users=test_clean_data.map( lambda x: int(x[0])).collect()


grouped_rates = train_clean_data.filter(lambda x: x[0] in test_users).map(lambda x: (x[0],x[1])).groupByKey().map(lambda x: (x[0], list(x[1]))).collect()
grouped_rates_dic = dict(grouped_rates)


item_ratings = train_clean_data.map(lambda x: (x[0], x[2])).aggregateByKey((0,0), lambda x,y: (x[0] + y, x[1] + 1),lambda x,y: (x[0] + y[0], x[1] + y[1]))
user_ratings_mean = item_ratings.mapValues(lambda x: (x[0] / (x[1]))).collect()
user_ratings_mean_dic=dict(user_ratings_mean)


item_ratings_forTop = train_clean_data.map(lambda x: (x[1], x[2])).aggregateByKey((0,0), lambda x,y: (x[0] + y, x[1] + 1),lambda x,y: (x[0] + y[0], x[1] + y[1]))#.sortBy(lambda x: x[1][1], ascending=False)
shrinkage_factor = 5
item_ratings_mean = item_ratings_forTop.mapValues(lambda x: (x[0] / (x[1] + shrinkage_factor))).sortBy(lambda x: x[1], ascending = False).map(lambda x: x[0]).collect()


#matrix interactions with global effect creation
users = train_clean_data.map(lambda x: x[0]).collect()
items = train_clean_data.map(lambda x: x[1]).collect()
ratings = train_clean_data.map(lambda x: x[2]).collect()
UxI_basic= sm.csr_matrix((ratings, (users, items)))

#remove global effect
global_mean = train_clean_data.map(lambda x: x[2]).mean()

trainr2 = train_clean_data.map(lambda x: (x[0], x[1], x[2]-global_mean))
temp = trainr2.map(lambda x: (x[1],(x[2],1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).mapValues(lambda x: x[0]/x[1])
i_Bias = temp.collectAsMap()

trainr3 = trainr2.map(lambda x: (x[0],x[1], x[2] - i_Bias.get(x[1],0)))
temp2 = trainr3.map(lambda x: (x[0],(x[2],1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).mapValues(lambda x: x[0]/x[1])
u_Bias = temp2.collectAsMap()

train_clean_data = train_clean_data.map(lambda x: (x[0], x[1], x[2] - i_Bias.get(x[1],0) - u_Bias.get(x[0],0)))
train_clean_data.take(5)

#matrix interactions without global effect creation
users = train_clean_data.map(lambda x: x[0]).collect()
items = train_clean_data.map(lambda x: x[1]).collect()
ratings = train_clean_data.map(lambda x: x[2]).collect()
UxI_global1= sm.csr_matrix((ratings, (users, items)))

#matrix icm creations
items_for_features= icm_clean_data.map(lambda x:x[0]).collect()
features = icm_clean_data.map(lambda x:x[1]).collect()
items_for_features.append(37142)
features.append(0)
ones=[1]*len(items_for_features)
IxF= sm.csr_matrix((ones, (items_for_features, features)))

In [3]:
n_users,n_items=UxI_basic.shape
n_features=IxF.shape[1]

In [ ]:
'''SVD'''
K = 870
%time U, S, Vt = sparsesvd(UxI_basic.tocsc(), K)


In [ ]:
S=np.sqrt(S.data)
S=np.diag(S)

In [ ]:
#calc predictions
%time UxI_pred_SVD=sm.csr_matrix(U.T.dot(S).dot(Vt))

In [4]:
'''content based'''
IDF=[0]*n_features
for i in tqdm(range(n_features)):
    IDF[i]=np.log10(n_items/len(IxF.getcol(i).nonzero()[1]))
%time IxF=normalize(IxF,axis=1)
%time IxF_idf=IxF.multiply(IDF)
%time UxF=UxI_global1.dot(IxF_idf)
%time UxI_pred_CB=UxF.dot(IxF.T)


CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.33 ms
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 5.12 ms
CPU times: user 56 ms, sys: 0 ns, total: 56 ms
Wall time: 58.1 ms


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


CPU times: user 8.17 s, sys: 1.04 s, total: 9.21 s
Wall time: 9.21 s


In [ ]:
del IxF

In [ ]:
'''collaborative filtering item based via content'''
#calc similarities
%time IxI_sim=IxF_idf.dot(IxF_idf.T)
IxI_sim.setdiag(0)

In [ ]:
#take knn items
IxI_sim_knn=sm.lil_matrix((n_items,n_items))
k=200
for i in tqdm(range(n_items)):    
    top_k_idx =IxI_sim.getrow(i).toarray()[0].argpartition(-k)[-k:]
    IxI_sim_knn[i,top_k_idx]=IxI_sim[i,top_k_idx]

In [ ]:
del IxI_sim 

In [ ]:
#calc predictions
%time UxI_pred_CI=UxI_global1.dot(IxI_sim_knn.T)

In [ ]:
del IxI_sim_knn

In [ ]:
'''collaborative filtering user based via content'''
#calc similarities
%time UxU_sim=UxF.dot(UxF.T) #numerators of cosine

In [ ]:
%time cos=sm.csr_matrix(cosine_similarity(UxF)) #cosine
cos.data=1/cos.data
denominators=UxU_sim.multiply(cos) #get denominators of cosine
del cos

In [ ]:
denominators.data+=1 #add shrinkage to cosine denominator
denominators.data=1/denominators.data
%time UxU_sim=UxU_sim.multiply(denominators) #calc shrinked cosine
del denominators

In [ ]:
UxU_sim.setdiag(0)

In [ ]:
#take knn users
UxU_sim_knn=sm.lil_matrix((n_users,n_users))
k=100
for i in tqdm(range(n_users)):    
    top_k_idx =UxU_sim.getrow(i).toarray()[0].argpartition(-k)[-k:]
    UxU_sim_knn[i,top_k_idx]=UxU_sim[i,top_k_idx]  

In [ ]:
del UxU_sim

In [ ]:
#calc_predictions
%time UxI_pred_CU=UxU_sim_knn.dot(UxI_global1) #k=50->382

In [ ]:
del UxU_sim_knn
del UxI
del UxF
del IxF_idf

'''asdadasdasdasdasdasdasdadasdasdasdadasdadasdasdadasdad'''
''''''

In [ ]:
#create matrix for test users predictions
UxI_pred_CB_test=sm.lil_matrix((n_users,n_items))
UxI_pred_CI_test=sm.lil_matrix((n_users,n_items))
UxI_pred_CU_test=sm.lil_matrix((n_users,n_items))
UxI_pred_SVD_test=sm.lil_matrix((n_users,n_items))

In [ ]:
#take only test users predictions
%time UxI_pred_CB_test[test_users,:]=UxI_pred_CB[test_users,:]
%time UxI_pred_CI_test[test_users,:]=UxI_pred_CI[test_users,:]
%time UxI_pred_CU_test[test_users,:]=UxI_pred_CU[test_users,:]
%time UxI_pred_SVD_test[test_users,:]=UxI_pred_SVD[test_users,:]

In [ ]:
#remove items already voted
for user in tqdm(test_users):
    UxI_pred_CB_test[user,grouped_rates_dic[user]]=0
    UxI_pred_CI_test[user,grouped_rates_dic[user]]=0
    UxI_pred_CU_test[user,grouped_rates_dic[user]]=0
    UxI_pred_SVD_test[user,grouped_rates_dic[user]]=0

In [ ]:
#convert to csr to improve future operations performances
%time UxI_pred_CB_test=UxI_pred_CB_test.tocsr()
%time UxI_pred_CI_test=UxI_pred_CI_test.tocsr()
%time UxI_pred_CU_test=UxI_pred_CU_test.tocsr()
%time UxI_pred_SVD_test=UxI_pred_SVD_test.tocsr()

In [ ]:
#rescale algoritms
%time UxI_pred_CB_test/=UxI_pred_CB.max()
%time UxI_pred_CI_test/=UxI_pred_CI.max()
%time UxI_pred_CU_test/=UxI_pred_CU.max()
%time UxI_pred_SVD_test/=UxI_pred_SVD.max()

Predictions of all algorithms computed, now have fun with hybrids!

In [ ]:
%time UxI_pred_1=UxI_pred_CB_test*0.80+UxI_pred_SVD_test*0.20

In [ ]:
%time UxI_pred_1/=UxI_pred_1.max()

In [ ]:
%time UxI_pred_2=UxI_pred_1*0.98+UxI_pred_CI_test*0.02

In [ ]:
%time UxI_pred_2/=UxI_pred_2.max()

In [ ]:
%time UxI_pred=UxI_pred_2*0.99+UxI_pred_CU_test*0.01

In [ ]:
%time UxI_pred/=UxI_pred.max()

In [ ]:
UxI_pred=UxI_pred_CI

In [ ]:
#f = open('submission_wSUM_CI-CU-08-02.csv', 'wt')
f = open('check.csv', 'wt')
writer = csv.writer(f)
writer.writerow(('userId','RecommendedItemIds'))

for user in tqdm(test_users):
    top=[]

    user_predictions=user_predictions=(-UxI_pred.getrow(user).toarray()[0]).argsort()
    i=0
    while len(top)<=5:
        prediction=user_predictions[i]
        
        if UxI_pred[user,prediction]==0.0:
            j=0
            while len(top)<=5:
                prediction = item_ratings_mean[j]
                while prediction in grouped_rates_dic[user] or prediction in top:
                    j += 1
                    prediction = item_ratings_mean[j]
                top.append(prediction)
                j += 1
        
        
        
        if prediction in grouped_rates_dic[user]:
            i+=1
            continue
        
        i+=1
        top.append(prediction)
        
   
    writer.writerow((user, '{0} {1} {2} {3} {4}'.format(top[0], top[1], top[2], top[3], top[4])))

f.close()

SOMMOZZI E BORDOZZO

In [ ]:
%time UxI_pred_1=UxI_pred_CB_test*0.80+UxI_pred_SVD_test*0.20

In [ ]:
%time UxI_pred_2=UxI_pred_CI_test*0.80+UxI_pred_SVD_test*0.20

In [ ]:
def bordaLikeAggr(rank1,rank2):    
    result=[0]*n_items
    rg=150
    for i in range(rg):
        item1=rank1.argmax()
        item2=rank2.argmax()        
        
        if rank1[item1]>0.0:
            result[item1]+=(0.7/(i+1))
        rank1[item1]=-9

        if rank2[item2]>0.0:
            result[item2]+=(0.3/(i+1))
        rank2[item2]=-9
    
    return sm.csr_matrix(result)

In [ ]:
f = open('submission_bordalile_sum-CB-SVD-80-20_CI-SVD-80-20_70-30.csv', 'wt')
writer = csv.writer(f)
writer.writerow(('userId','RecommendedItemIds'))
for user in tqdm(test_users):
    top=[0,0,0,0,0]

    user_predictions=bordaLikeAggr(UxI_pred_1.getrow(user).toarray()[0],UxI_pred_2.getrow(user).toarray()[0])
    iterator = 0
    for i in range(5):
        prediction = user_predictions.argmax()
        while prediction in grouped_rates_dic[user] and prediction != 0:
            user_predictions[0,prediction]=-9
            prediction=user_predictions.argmax()
        if prediction == 0:
            prediction = item_ratings_mean[iterator]
            while prediction in grouped_rates_dic[user] or prediction in top:
                iterator += 1
                prediction = item_ratings_mean[iterator]
            iterator += 1
        else:
            user_predictions[0,prediction]=-9
        top[i]=prediction    
    writer.writerow((user, '{0} {1} {2} {3} {4}'.format(top[0], top[1], top[2], top[3], top[4])))

f.close()

In [ ]:
UxI.min()